In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# import dataset

In [2]:
df = pd.read_csv('london_weather.csv')


# fill missing data

In [3]:
# df = df.fillna(1)
df = df.fillna(method='ffill')
# df_filled = df.interpolate(method='linear')



# split dataset

In [4]:
x = df.drop(columns=['max_temp','snow_depth'], axis=1)
y = df['max_temp']

x = np.array(x)
y = np.array(y)

In [5]:
from sklearn.model_selection import train_test_split

x, x_test_main, y, y_test_main = train_test_split(x, y, test_size=0.1, random_state=42)
# y_train = y_train.astype(int)
# y_test = y_test.astype(int)
y_test_main.shape

(1535,)

# normalizing

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
x_test_main = scaler.fit_transform(x_test_main)
x[0]

array([0.36609191, 0.33333333, 0.86875   , 0.82352941, 0.67213115,
       0.6568915 , 0.        , 0.79683973])

# pca

In [7]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=8)

# x = pca.fit_transform(X=x)
# x_test_main = pca.transform(X=x_test_main)


# define model 1

In [8]:


# model = keras.Sequential()
# model.add(keras.layers.Input(shape=(8,)))
# model.add(keras.layers.Dense(units=32, activation='relu'))
# model.add(keras.layers.Dense(units=32, activation='sigmoid'))
# model.add(keras.layers.Dense(units=1))
# model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
# # model.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.mean_squared_error, metrics=['accuracy'])
# model.fit(x, y, epochs=20, batch_size=64, validation_data=(x_test_main, y_test_main))
# y_pred = model.predict(x_test_main)
# mse = mean_squared_error(y_true=y_test_main, y_pred=y_pred)
# mse
# # 12.187250703339979 => mean_squared_error with scaler and pca
# # 10.015396159320842 => mean_squared_error with scaler and without pca
# # 42.39169921336769 => mean_squared_error without scaler and without pca
# # 10.933645420023264 => mean_squared_error without column snow_depth and without pca and with scaler

# define model 2

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=6)
def model_creator():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(8,)))
    model.add(keras.layers.Dense(units=448, activation='relu'))
    model.add(keras.layers.Dense(units=320, activation='relu'))
    model.add(keras.layers.Dense(units=32, activation='relu'))
    model.add(keras.layers.Dense(units=1, activation='linear'))
    model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])
    return model



mse_list = []
for train, validation in kf.split(x, y):

    # x_train = x_train_main[train]
    # y_train = y_train_main[train]

    # x_test = x_train_main[validation]
    # y_test = x_train_main[validation]

    # x_train, x_test = x[train], x[validation]
    # y_train, y_test = y[train], y[validation]

    x_train = x[train]
    x_test = x[validation]
    y_train = y[train]
    y_test = y[validation]

    model = model_creator()
    model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test), verbose=0)

    y_pred = model.predict(x_test)

    mse_list.append(mean_squared_error(y_test, y_pred))

# mse_list

y_pred_main = model.predict(x_test_main)
mse_main = mean_squared_error(y_test_main, y_pred_main)
mse_main, np.mean(mse_list)




# # 6.304961665296364 => mean_squared_error with scaler and without pca
# #(7.039170778006721, 5.531378233763437) => mean_squared_error with random search parameters
# (6.022063032004108, 5.339298581649459)


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


(5.365472610540216, 4.892028082248992)

# best parameters model 1

In [10]:
# ! pip install keras-tuner

In [11]:
# from keras_tuner import RandomSearch
# # from keras_tuner import GridSearch

# # Define your model inside a function with hyperparameters as input
# def build_model(hp):
#     model = keras.Sequential()

#     # Tune number of layers (from 2 to 4)
#     for i in range(hp.Int('num_layers', 2, 4)):
#         model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
#                                      activation=hp.Choice('activation_' + str(i), ['relu', 'tanh'])))

#     model.add(keras.layers.Dense(1, activation='softmax'))

#     # Tune optimizer learning rate
#     model.compile(
#         optimizer=keras.optimizers.Adam(
#             hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
#         ),
#         loss='mean_squared_error',
#         metrics=['mean_squared_error'])

#     return model

# # Create the tuner object
# tuner = RandomSearch(
#     build_model,
#     objective='val_mean_squared_error', # Metric to optimize
#     max_trials=5,             # Number of random configurations to try
#     executions_per_trial=3,    # Number of models to train for each trial (for robustness)
#     directory='my_dir',        # Directory to save logs and checkpoints
#     project_name='random_search'
#     # project_name='grid_search'
# )



# # Start the search
# tuner.search(x, y, epochs=10, validation_data=(x_test_main, y_test_main))

# # Get the best hyperparameters and the best model
# best_hps = tuner.get_best_hyperparameters()[0]
# best_model = tuner.get_best_models()[0]

# # Summary of the best model
# best_model.summary()

# # print(f"Best number of units: {best_hps.get('units')}")
# # print(list(best_hps))

In [12]:
# random search model:

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
# ┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
# ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
# │ dense (Dense)                        │ (None, 448)                 │           4,032 │
# ├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
# │ dense_1 (Dense)                      │ (None, 320)                 │         143,680 │
# ├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
# │ dense_2 (Dense)                      │ (None, 32)                  │          10,272 │
# ├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
# │ dense_3 (Dense)                      │ (None, 1)                   │              33 │
# └──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
#  Total params: 158,017 (617.25 KB)
#  Trainable params: 158,017 (617.25 KB)
#  Non-trainable params: 0 (0.00 B)

# best parameters model 2

In [13]:
# from sklearn.model_selection import RandomizedSearchCV
# from keras.wrappers.scikit_learn import KerasClassifier

# # Define a function to create your Keras model
# def create_model(activation='relu', hidden_layer_size=32, dropout_rate=0.2):
#     # ... (Your model architecture)

#     return model

# # Create a KerasClassifier instance
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define the hyperparameter search space
# param_dist = {
#     'activation': ['relu', 'tanh', 'sigmoid'],
#     'hidden_layer_size': [16, 32, 64, 128],
#     'dropout_rate': [0.2, 0.3, 0.4, 0.5]
# }

# # Create a RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3)

# # Fit the random search to your data
# random_search.fit(X_train, y_train)

# # Get the best parameters and model
# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# best parameters model 3

In [14]:
# def get_reg(meta, hidden_layer_sizes, dropout, active='relu'):
#     n_features_in_ = meta["n_features_in_"]
#     model = keras.models.Sequential()
#     model.add(keras.layers.Input(shape=(n_features_in_,)))
#     for hidden_layer_size in hidden_layer_sizes:
#         model.add(keras.layers.Dense(hidden_layer_size, activation=active))
#         model.add(keras.layers.Dropout(dropout))
#     model.add(keras.layers.Dense(1))
#     return model
# from scikeras.wrappers import KerasRegressor


# reg = KerasRegressor(
#     model=get_reg,
#     loss="mse",
#     metrics=[keras.metrics.R2Score],
#     hidden_layer_sizes=(100,),
#     dropout=0.5,
# )
# reg.fit(x_train, y_train)
# from sklearn.model_selection import RandomizedSearchCV

# param_grid = {
#     'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],
#     'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
# }
# rs = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, cv=3, n_iter=20)

# rs.fit(x_train, y_train)


# best parameters model 4

In [15]:
# ! pip install tensorflow scikit-learn

In [16]:
# # Required libraries
# import numpy as np
# from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from tensorflow.keras.optimizers import Adam

# # Function to create Keras model
# def create_model(optimizer='adam', neurons=32, activation='relu'):
#     # Define the model
#     model = Sequential()
#     model.add(Dense(neurons, input_dim=10, activation=activation))  # Example input_dim=10
#     model.add(Dense(1))  # Output layer (regression)

#     # Compile the model
#     model.compile(optimizer=optimizer, loss='mean_squared_error')
#     return model

# # Wrap the model using KerasRegressor
# keras_regressor = KerasRegressor(build_fn=create_model, verbose=0)

# # Define the hyperparameter grid to search
# param_grid = {
#     'batch_size': [10, 20],
#     'epochs': [10, 50],
#     'optimizer': ['adam', 'rmsprop'],
#     'neurons': [16, 32, 64],
#     'activation': ['relu', 'tanh'],
# }

# # Create the GridSearchCV object
# grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)



# # Fit the grid search
# grid_result = grid.fit(x, y)

# # Summarize results
# print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
